# Introduction à l'apprentissage automatique - Examen


<br>

### Reconnaissance du genre par analyse de la parole

<br>

La base de donnée [que vous devez télécharger ici](https://members.loria.fr/FSur/enseignement/apprauto/voice2.csv) est construite à partir de 1584 échantillons de voix collectés auprès d'orateurs femmes ou hommes. Chaque échantillon subit une analyse spectrale. Dans la base de données, chaque observation est décrite par les caractéristiques suivantes:

   * meanfreq: mean frequency (in kHz)
   * sd: standard deviation of frequency
   * median: median frequency (in kHz)
   * Q25: first quantile (in kHz)
   * Q75: third quantile (in kHz)
   * IQR: interquantile range (in kHz)
   * skew: skewness 
   * kurt: kurtosis 
   * label: male (1) or female (0)

Vous pouvez ouvrir le fichier `voice2.csv` avec Excel ou un éditeur de texte pour observer les données.

_Remarque_: pour réussir cet exercice il n'est pas nécessaire d'avoir compris comment les caractéristiques sont extraites d'un échantillon de voix.

<br>

### Consignes

Seul votre code python doit figurer dans ce carnet Jupyter. Vos réponses doivent être rédigées sur une copie.

_Rappel_: le fichier `.ipynb` et le jeu de données `voice2.csv` doivent être dans le même répertoire de travail, et vous devez lancer le serveur `jupyter notebook` dans ce répertoire de manière à ce que vos modifications soient sauvegardées dans le fichier `.ipynb`.

Sauvegardez régulièrement votre travail.

A la fin du test, faites `Noyau / Redémarrer & effacer les sorties` (barre de commandes en haut du carnet) de manière à générer un fichier `.ipynb` de taille raisonnable, et sauvegardez votre travail. Envoyez votre fichier à l'adresse mail sur@loria.fr.

<br>

### Travail demandé

Le but de l'exercice est de prédire si un échantillon correspond à une voix d'homme ou de femme à partir des 8 caractéristiques fournies. 

La cellule suivante permet de stocker les vecteurs de caractéristiques et les labels (0 / 1), crée une base d'apprentissage et une base de test de même taille, et normalise les données.

In [ ]:
from sklearn import preprocessing, neighbors, model_selection, neural_network, svm, linear_model, metrics
import matplotlib.pyplot as plt
import numpy as np

# tableau des vecteurs de caractéristiques
X = np.genfromtxt('voice2.csv',usecols=(0,1,2,3,4,5,7),skip_header=1,delimiter=';')
# vecteur des labels 0/1
Y = np.genfromtxt('voice2.csv',usecols=(8),skip_header=1,delimiter=';')

print("nombre d'observations: %d  -  nombre de caractéristiques par observation: %d" % (X.shape[0],X.shape[1]))

X_tr, X_te, Y_train, Y_test = model_selection.train_test_split(X,Y,test_size=.5)

X_train = preprocessing.StandardScaler().fit_transform(X_tr)
X_test = preprocessing.StandardScaler().fit(X_tr).transform(X_te)

<br>

Proposez un classifieur permettant de prédire si un échantillon de voix provient d'un homme ou d'une femme.

Vous testerez différents classifieurs dont vous fixerez le ou les hyperparamètre(s) par une méthode que vous préciserez. Vous calculerez un score de classification et la matrice de confusion. Vous expliquerez aussi en quoi consistent ces deux indicateurs.

Pourquoi l'entraînement et le test des classifieurs se font-ils sur deux jeux de données différentes?

# Début rédaction


## Questions de cours.

Le score de classification indique la proportions de valeurs correctement prédites par le classifieur.

La matrice de confusion en i, j indique la quantité d'échantillons appartenant à la classe i ayant été prédits comme appartenants à la classe j.

La base de test est différente de la base d'entrainement pour éviter le biais de sur-apprentissage lors de l'évaluation du classifieur (classifieur qui apprend "par coeur").

## SVM

On fait d'abord une première évaluation du classifieur "naïve", en choisissant arbitrairement les hyper-paramètres "gamma" et  "C". Le noyau choisis est "rbf", le noyau gaussien.

In [ ]:
clf = svm.SVC(C=0.1, gamma='scale', kernel = "rbf")    
clf.fit(X_train, Y_train)

print("score SVM %.2f" % clf.score(X_test, Y_test) )
print("nombre de vecteurs supports : ", clf.n_support_)

On a alors un score de prédiction de 0.9 sur la base de test.
On va ensuite chercher à afficer ces paramètres, a l'aide de la fonction "gridsearch".

In [ ]:
gamma_range=10**(np.arange(-3.,4.5,.5))
C_range=10**(np.arange(-3.,3.5,.5)) 
parameters = { 'gamma': gamma_range, 'C':C_range }
SVM = svm.SVC(kernel='rbf')
gridsearch=model_selection.GridSearchCV(SVM, parameters)
gridsearch.fit(X_train,Y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch.best_estimator_)
print("Meilleurs paramètres:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.colorbar()
plt.show();

Le meilleur couple de valeurs trouvé est : C = 100, gamma = 0.032. Regardons maintenant le meilleur noyau pour ce couple.

In [ ]:
kernels = ['linear', 'poly', 'rbf', 'sigmoid']

for kernel in kernels : 
    clf = svm.SVC(C=100, gamma=0.032, kernel = kernel)    
    clf.fit(X_train, Y_train)

    print("score SVM : %.2f" % clf.score(X_test, Y_test), "pour le noyau ", kernel )

Il semble donc que le noyau 'rbf' soit le meilleur. Pour en être sûr, je réeffectue un gridsearch avec le moins bon noyau (sigmoid) pour voir si les valeurs varient fortement.

In [ ]:
gamma_range=10**(np.arange(-3.,4.5,.5))
C_range=10**(np.arange(-3.,3.5,.5)) 
parameters = { 'gamma': gamma_range, 'C':C_range }
SVM = svm.SVC(kernel='sigmoid')
gridsearch=model_selection.GridSearchCV(SVM, parameters)
gridsearch.fit(X_train,Y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch.best_estimator_)
print("Meilleurs paramètres:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(C_range),len(gamma_range))
plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('gamma')
plt.ylabel('C')
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.colorbar()
plt.show();

Les valeurs sont modifiées, mais on reste sur un score inférieeur à 'rbf'

Pour ce qui est de la matrice de confusion :

In [ ]:
from sklearn import metrics

clf = svm.SVC(C=100, gamma=0.032, kernel = 'rbf')    
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)

print("score MLP %.4f" % clf.score(X_test, Y_test) )
print(metrics.confusion_matrix(Y_test,Y_test_pred))

## MLP à une couche.

on test ici la classification par un perceptron à une couche cachée (20 neurones). On cherchera les bonnes valeurs pour les hyperparamètres alpha et max_iter.

Avec relu en fonction d'activation :

In [ ]:
from sklearn.neural_network import MLPClassifier

alpha_range=10**(np.arange(-4.8,-3.2,.1))
max_iter_range=10**(np.arange(2,2.7,.1)) 
# alpha_range=np.arange(0.00005,0.00015,0.0001)
# max_iter_range=np.arange(100,500,50)
parameters = { 'alpha': alpha_range, 'max_iter':max_iter_range }
SVM = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20), random_state=1, activation = 'relu')
gridsearch=model_selection.GridSearchCV(SVM, parameters)
gridsearch.fit(X_train,Y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch.best_estimator_)
print("Meilleurs paramètres:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(max_iter_range),len(alpha_range))
# plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('alpha')
plt.ylabel('max_iter')
plt.xticks(np.arange(len(alpha_range)), alpha_range, rotation=45)
plt.yticks(np.arange(len(max_iter_range)), max_iter_range)
plt.colorbar()
plt.show();

avec la sigmoide :

In [ ]:
alpha_range=10**(np.arange(-5.2,-3.2,.1))
max_iter_range=10**(np.arange(2,2.7,.1)) 
# alpha_range=np.arange(0.00005,0.00015,0.0001)
# max_iter_range=np.arange(100,500,50)
parameters = { 'alpha': alpha_range, 'max_iter':max_iter_range }
SVM = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(20), random_state=1, activation = 'logistic')
gridsearch=model_selection.GridSearchCV(SVM, parameters)
gridsearch.fit(X_train,Y_train)
print("Meilleur estimateur trouvé:")
print(gridsearch.best_estimator_)
print("Meilleurs paramètres:")
print(gridsearch.best_params_)

scores = gridsearch.cv_results_['mean_test_score'].reshape(len(max_iter_range),len(alpha_range))
# plt.figure(figsize=[10,10])
plt.imshow(scores)
plt.xlabel('alpha')
plt.ylabel('max_iter')
plt.xticks(np.arange(len(alpha_range)), alpha_range, rotation=45)
plt.yticks(np.arange(len(max_iter_range)), max_iter_range)
plt.colorbar()
plt.show();

Les scores obtenus sont similaires, on garde la sigmoide, avec alpha =  0.0004 et max_iter = 250.

In [ ]:
from sklearn import metrics

clf = MLPClassifier(solver='lbfgs', alpha=4e-4, max_iter = 250, hidden_layer_sizes=(20), random_state=1, activation = 'logistic')
  
clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)

print("score MLP %.4f" % clf.score(X_test, Y_test) )
print(metrics.confusion_matrix(Y_test,Y_test_pred))

Ce classifieur semble sensiblement moins performant que la machine a vecteur support, on peut voir qu'il y a 10 échantillons mal classés supplémentaires.

## Plus proches voisins.

on teste enfin la méthode des plus proches voisins. On cherchera a determiner le bon nombre de voiosinns à considérer.

In [ ]:
results = []

for i in range(1,20):
    knn = neighbors.KNeighborsClassifier(n_neighbors=i)
    results.append([i, knn.fit(X_train, Y_train).score(X_test, Y_test)])

plt.plot([results[i][0] for i in range(len(results))], [results[i][1] for i in range(len(results))], 'ro')
plt.axis([0, 20, 0.90, 0.95])
plt.show()
    
for k in results:
    print(k)

Le meilleur résultat est obtenu avec 3 plus proches voisins.

In [ ]:
clf = neighbors.KNeighborsClassifier(n_neighbors=3)

clf.fit(X_train, Y_train)
Y_test_pred = clf.predict(X_test)

print("score MLP %.4f" % clf.score(X_test, Y_test) )
print(metrics.confusion_matrix(Y_test,Y_test_pred))

On obtient des résultats à peu près équivalents aux deux autres classifieurs : plus de 0.93

Cependant, on peut voir que la matrice de confusion est différente : les erreurs de prédiction des échantillons de la classe 1 sont plus faibles que ceux de la classe 2.

## Conclusion.

Pour ce problème, je choisirai une machine à vecteurs supports, avec pour paramètres :

- kernel = 'rbf'
- C = 100 
- gamma = 0.032
